In [1]:
#import libraries
import pandas as pd
import numpy as np
import nltk 
from nltk.classify.textcat import TextCat 
from matplotlib.pyplot import plot 
#from datascience import *

In [2]:
# Give me more KERNEL SPACE!jupyter notebook --NotbookApp.iopub_Data_Rate_Limit=1e10

In [2]:
!jupyter notebook list

Currently running servers:
http://0.0.0.0:8888/user/emilyxu/ :: /home/jovyan


In [3]:
!jupyter notebook

In [2]:
!jupyter notebook --generate-config

Writing default config to: /home/jovyan/.jupyter/jupyter_notebook_config.py


In [4]:
!jupyter notebook --NotebookApp.max_buffer_size=10000000000

In [2]:
#fuzzywuzzy
!pip3 install fuzzywuzzy[speedup]
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#(py)torch
#!pip install torch

#langid (language identification)
#!pip install langid
#import langid

In [4]:
#DO NOT RUN 
#This code creates a traditional table as seen in the datascience * package but takes up too much memory. 
#I suggest running the code block after this one that generates chunks of 10 for each row of data.

chatbots_data = Table.read_table("chatbots_data.csv")

#Convert Pandas dataframe to Table (where you can use table.select(...)). Not suggested -- takes up lots of memory
chatbots_data = Table.from_df(chatbots_data_chunks, keep_index=False)

chatbots_data

In [3]:
#dividing the data into chunks of 10, because of memory issues (kernel keeps dying)
#for example, this is what one chunk looks like
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    print(chunk.shape)
    print("="*66)
    print(chunk.head(2))
    print("="*66)
    break
    
#appending chunks to chatbot_list, an empty array
chatbot_list = []
chunksize = 10
for chunk in pd.read_csv("chatbots_data.csv", chunksize = chunksize):
    chatbot_list.append(chunk)

#Combine all the chunks into one dataframe
chatbots_data_chunks = pd.concat(chatbot_list, axis=0)
chatbots_data_chunks = pd.DataFrame(data = chatbots_data_chunks)

(10, 7)
              Created On     Flow Name  Merged Code  \
0  5/27/2020 10:31:06 PM    OB_NewUser          NaN   
1   5/27/2020 7:38:52 PM  HLP_Feedback          NaN   

                                             Message    Org Name  \
0                                            Kya kru   Chhaa Jaa   
1  I like you\nAnd I think improve not only about...  Big Sis V3   

   Referred Flow                                  Uuid  
0            NaN  915617a9-c0ae-47fc-8df0-54d72d4f41d9  
1            NaN  148d099c-54eb-4182-961c-068eeda96691  


In [4]:
#cleans the "Message" column
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"").replace('_',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

chatbots_data_chunks['Message']=chatbots_data_chunks['Message'].map(lambda s:preprocess(s)) 

In [3]:
#view chunks
chatbots_data_chunks

In [5]:
#View cleaned data, all 80,000+ rows

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
chatbots_data_chunks

In [5]:
#collocation finder....might scrap later
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(chatbots_data_chunks["Message"].values.reshape(-1, ))

#bigrams that appear 3+ times
finder.apply_freq_filter(2)

return the 10 n-grams with the highest PMI (pointwise mutual information)
finder.nbest(bigram_measures.pmi, 10)

In [5]:
#Backend portion...find the most common phrases 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

vectorizer = CountVectorizer(ngram_range = (3,3)) 
X1 = vectorizer.fit_transform(chatbots_data_chunks["Message"])  
features = (vectorizer.get_feature_names()) 
#print("\n\nFeatures : \n", features) <-- if you want to see all the phrases
#print("\n\nX1 : \n", X1.toarray())   <-- but I don't recommend running
  
# Applying TFIDF 
vectorizer = TfidfVectorizer(ngram_range = (3,3)) 
X2 = vectorizer.fit_transform(chatbots_data_chunks["Message"]) 
scores = (X2.toarray()) 
#print("\n\nScores : \n", scores) <--- I don't recommend running

In [8]:
# Getting bottom 100 ranking features 
sums = X2.sum(axis = 0) 
data1 = [] 
for col, term in enumerate(features): 
    data1.append( (term, sums[0,col] )) 
ranking = pd.DataFrame(data1, columns = ['term','rank']) 
words = (ranking.sort_values('rank', ascending = True)) 
# Alternatively, put "False" here to see the most common trigrams

words.head(500)

,term,rank
9421,der try tug,0.071186
44145,try tug juju,0.071186
44141,try try trees,0.071186
44140,try try tffff,0.071186
44139,try trees dressed,0.071186
...,...,...
47119,warren images nwith,0.130189
45650,virus thrives causes,0.130189
35300,seattle nursing home,0.130189
43846,transmission nhumans reservoirs,0.130189


In [10]:
#backend part that allows fuzzy to work
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

choices = chatbots_data_chunks['Message'].unique()

In [2]:
# much services cost
# it appears that users are curious about the services the bots provide, as well as how much those services might cost
# however, fuzzywuzzy is also pulling "STIs" for "services" which could be an issue
process.extract("much services cost", choices, limit = 10, scorer = fuzz.token_sort_ratio)

In [11]:
#change mind sex
process.extract("change mind sex", choices, limit = 10, scorer = fuzz.token_sort_ratio)

In [12]:
#know ready sex
process.extract("know ready sex", choices, limit = 10, scorer = fuzz.token_sort_ratio)

In [14]:
nltk.download('crubadan')
#N-Gram based categorization, identifying languages...which is very off...

tc = TextCat()
chatbots_data_chunks["Message"][0:5].apply(tc.guess_language)

In [11]:
import numpy as np
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn
import sparse_dot_topn.sparse_dot_topn as ct

Processing ./.cache/pip/wheels/4a/64/25/2e1cedbf7513fecda8d396a0431f3a1e217f45f1ab565e8ce8/sparse_dot_topn-0.2.9-cp38-cp38-linux_x86_64.whl


In [12]:
# backend portion for string matching
from sklearn.feature_extraction.text import TfidfVectorizer

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

message = chatbots_data_chunks["Message"]
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(message)

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
    idx_dtype = np.int32
    nnz_max = M*ntop
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1

def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

In [13]:
#string-matching output
matches_df = get_matches_df(matches, message, top=100000)
matches_df = matches_df[matches_df['similarity'] < 0.99999] # Remove all exact matches
matches_df.sample(50)

,left_side,right_side,similarity
80363,love know someone love,know someone loves,0.847443
25930,give tips,give tips sex,0.838539
722,chup,chupp,0.844864
69169,shy communicate partner,communicate partner,0.867580
93225,iam worthy respect,worthy respect,0.825499
21698,send sex nsend sex videos,send sex videos,0.859452
51580,confront boyfriend cheating,know boyfriend cheating confront,0.857345
44623,aapka naam kya hai,aapka naam kya hai apna naam batao,0.819018
97041,satisfy man sex,satisfy man,0.898854
71212,problems,problem,0.873137


In [8]:
# Use google translate's algorithm to identify languages, tag each n-gram with their corresponding language
# It should be easier to identify slang once google's algorithm sorts out language translations
# Dig around for the words that are not recognized by the algorithm using some sort of metric...perhaps those will be slang?

!python -m spacy download en
!pip install googletrans
!pip install spacy-langdetect

In [9]:
# Example function
import spacy

from spacy_langdetect import LanguageDetector
nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
text = "This is English text. Er lebt mit seinen Eltern und seiner Schwester in Berlin."
doc = nlp(text)

# document level language detection
print(doc._.language)
# sentence level language detection
for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

{'language': 'de', 'score': 0.9999964735841921}
This is English text. {'language': 'en', 'score': 0.9999976179869824}
Er lebt mit seinen Eltern und seiner Schwester in Berlin. {'language': 'de', 'score': 0.9999972511167439}


In [28]:
# What I want to do is take the messiest data that doesn't match anything and apply to language algorithm

uncommon_phrases_str = words['term'].head(500).astype(str).values.tolist()
uncommon_phrases_str

['der try tug',
 'try tug juju',
 'try try trees',
 'try try tffff',
 'try trees dressed',
 'tug ggg ggg',
 'tug juju juju',
 'dressed hits defy',
 'try der try',
 'hugh hugh ggg',
 'try tug ggg',
 'defy try try',
 'ggg try der',
 'trees dressed hits',
 'hits defy try',
 'hgh hugh hugh',
 'try try tug',
 'juju ggg try',
 'juju juju ggg',
 'juju juju juju',
 'dvd dvd beg',
 'cia fed cbbc',
 'dvd beg suburbs',
 'fbfvv greg dvd',
 'greg dvd dvd',
 'beg suburbs dvd',
 'cubeb uheegev gbcuuehfbbbbbsbbbsbsbsbbbbdbsbdbdhsdhdbeudjdbbhdhdhxvdbbdxhfgdhdhx',
 'dvd dvd dvd',
 'fed cbbc shzhzhzhxbxbxcbbccbbxbxbdbx',
 'dvd judge fbfvv',
 'via via cia',
 'cbbc van dvd',
 'via cia fed',
 'bbc via via',
 'suburbs dvd cubeb',
 'cbbc shzhzhzhxbxbxcbbccbbxbxbdbx cbbc',
 'shzhzhzhxbxbxcbbccbbxbxbdbx cbbc van',
 'judge fbfvv greg',
 'dvd cubeb uheegev',
 'van dvd judge',
 'box bbc band',
 'gag box bbc',
 'amiri nbillie jean',
 'heaven nwait wait',
 'billie jean nchristian',
 'way rock nshe',
 'way move nshe'

In [32]:
# Not sure if this data is the most important, I need to filter noise...

uncommon_phrases_str = '. '.join(uncommon_phrases_str)

In [16]:
# Trying to apply the function above to our chatbot data after converting the dataframe to a string

nlp = spacy.load("en")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
text = uncommon_phrases_str

doc = nlp(text)

In [30]:
# Function is breaking up phrases in the wrong area

print(doc._.language)

for i, sent in enumerate(doc.sents):
    print(sent, sent._.language)

{'language': 'en', 'score': 0.9999958090462673}
der try tug.try tug {'language': 'en', 'score': 0.9999939703495575}
juju.try try trees.try try {'language': 'en', 'score': 0.9999956466273108}
tffff.try trees dressed.tug ggg ggg.tug juju {'language': 'no', 'score': 0.9999962835026439}
juju.dressed hits {'language': 'et', 'score': 0.999998170304683}
defy.try der {'language': 'cy', 'score': 0.9999974739304596}
try.hugh hugh ggg.try tug ggg.defy try try.ggg {'language': 'cy', 'score': 0.7142826516461688}
try {'language': 'vi', 'score': 0.5714287870883022}
der.trees dressed {'language': 'af', 'score': 0.7142826848064792}
hits.hits {'language': 'en', 'score': 0.9999955357809618}
defy try.hgh {'language': 'cy', 'score': 0.9999956665348697}
hugh {'language': 'sw', 'score': 0.9999956190697695}
hugh.try try {'language': 'vi', 'score': 0.7142826025865403}
tug.juju {'language': 'hr', 'score': 0.9999947297257981}
ggg try.juju juju ggg.juju {'language': 'hr', 'score': 0.9999971142331605}
juju juju.dv

In [21]:
uncommon_phrases_dat = words['term'].head(500)
uncommon_phrases_dat

9421                         der try tug
44145                       try tug juju
44141                      try try trees
44140                      try try tffff
44139                  try trees dressed
                      ...               
47119                warren images nwith
45650               virus thrives causes
35300               seattle nursing home
43846    transmission nhumans reservoirs
47383               wednesday march near
Name: term, Length: 500, dtype: object

In [37]:
# visualize with principal component analysis / clustering with k means
# classify with logistic regression.

import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

# Are there too many 0's? seems to be a lot, and say some 2's in there for whatever reason...
vectors = vectorizer.fit_transform(uncommon_phrases_dat.astype(str)).toarray()
vectors

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 